In [11]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from xbbg import blp


Skipped: could not import 'blpapi': No module named 'blpapi'

## Downloading Data for 3 Maturities

In [6]:
def download_option_data(tickers, expiration_dates):
    
    # empty array to store all the downloaded options
    all_options = []

    # get today's date for time to maturity calculation
    today = datetime.today()
    # getting the interest rate through the closing price of 13 Week Treasury Bill
    #fed_funds = yf.Ticker("^IRX").history(period="1d")['Close'].iloc[-1]  

    # iterating through the list of provided tickers
    for ticker in tickers:
        try:
            # fetch most recent 15-minute interval closing price fro spot price
            spot_price = yf.Ticker(ticker).history(period="1d", interval="5m")["Close"].iloc[-1]
        except Exception as e:
            # error handling for the spot price fetching
            print(f"Error fetching 5-min close price for {ticker}: {e}")
            spot_price = None  # Assign None if price fetch fails

        # for each expirations dates getting the option data
        for exp_date in expiration_dates:
            try:
                # convert expiration date to datetime datatype
                exp_datetime = datetime.strptime(exp_date, "%Y-%m-%d")
                time_to_maturity = (exp_datetime - today).days / 365.0  # Convert to years

                # option chain from yfinance
                option_chain = yf.Ticker(ticker).option_chain(exp_date)
                # sperating the calls and puts chains
                calls = option_chain.calls
                puts = option_chain.puts

                # add identifying columns
                calls["ticker"] = ticker
                calls["expiration"] = exp_date
                calls["optionType"] = "Call"
                calls["timeToMaturity"] = round(time_to_maturity, 6)
                calls["spotPrice"] = round(spot_price,6)

                puts["ticker"] = ticker
                puts["expiration"] = exp_date
                puts["optionType"] = "Put"
                puts["timeToMaturity"] = round(time_to_maturity, 6)
                puts["spotPrice"] = round(spot_price,6)

                # combine calls & puts
                all_options.append(calls)
                all_options.append(puts)

            except Exception as e:
                print(f"Error fetching options for {ticker} ({exp_date}): {e}")

    # combine all data into a single DataFrame
    if not all_options:
        return None

    options_data = pd.concat(all_options, ignore_index=True)
    # adding interest rate to all the option data
    options_data["interestRate"] = round(fed_funds, 6)

    # rearrange columns and only getting the necessary fields
    desired_columns = ["ticker", "expiration", "timeToMaturity", "strike", "optionType",
                       "lastPrice", "bid", "ask", "impliedVolatility", "inTheMoney", "spotPrice", "interestRate"]
    options_data = options_data[desired_columns]

    return options_data

In [7]:
# get option data for NVDA fro three different maturities
tickers = ["NVDA"]
expiration_dates = ["2025-03-21", "2025-04-17", "2025-05-16"]
options_data = download_option_data(tickers, expiration_dates)

Error fetching 5-min close price for NVDA: Too Many Requests. Rate limited. Try after a while.
Error fetching options for NVDA (2025-03-21): Too Many Requests. Rate limited. Try after a while.
Error fetching options for NVDA (2025-04-17): Too Many Requests. Rate limited. Try after a while.
Error fetching options for NVDA (2025-05-16): Too Many Requests. Rate limited. Try after a while.


In [9]:
import requests

session = requests.Session()

ticker = "AAPL"
data = yf.Ticker(ticker, session=session)
hist = data.history(period="1mo")
print(hist)

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [10]:
def download_option_data_xbbg(ticker):
    """Fetches option data for the first 3 expiration dates using xbbg (Bloomberg)."""

    # Step 1: Retrieve the first 3 expiration dates
    expiration_dates = blp.bdp(f"{ticker} Equity", "OPT_EXPIRE_DT_LIST")["OPT_EXPIRE_DT_LIST"]
    
    if expiration_dates is None or len(expiration_dates) == 0:
        print(f"No expiration dates found for {ticker}.")
        return None

    expiration_dates = sorted(expiration_dates)[:3]  # Select first 3 expirations

    # Step 2: Fetch spot price
    spot_price = blp.bdp(f"{ticker} Equity", "PX_LAST")["PX_LAST"].iloc[0]

    # Step 3: Fetch options for each expiration date
    all_options = []
    today = datetime.today()

    for exp_date in expiration_dates:
        exp_str = exp_date.strftime("%Y-%m-%d")  # Convert to string format
        time_to_maturity = (exp_date - today).days / 365.0  # Convert to years

        # Fetch all options for this expiration
        option_chain = blp.bds(f"{ticker} Equity", "OPT_CHAIN")
        
        if option_chain is None or option_chain.empty:
            print(f"No option data for {ticker} ({exp_str})")
            continue

        # Process calls and puts
        for idx, row in option_chain.iterrows():
            option_type = "Call" if row["Put Call"] == "Call" else "Put"
            option_data = {
                "ticker": ticker,
                "expiration": exp_str,
                "timeToMaturity": round(time_to_maturity, 6),
                "strike": row["Strike"],
                "optionType": option_type,
                "lastPrice": row["Last Price"],
                "bid": row["Bid"],
                "ask": row["Ask"],
                "impliedVolatility": row["Implied Volatility"],
                "inTheMoney": row["In The Money"],
                "spotPrice": round(spot_price, 6),
            }
            all_options.append(option_data)

    # Combine all data into a DataFrame
    if not all_options:
        return None

    options_data = pd.DataFrame(all_options)

    # Select relevant columns
    desired_columns = ["ticker", "expiration", "timeToMaturity", "strike", "optionType",
                       "lastPrice", "bid", "ask", "impliedVolatility", "inTheMoney", "spotPrice"]
    options_data = options_data[desired_columns]

    return options_data

In [ ]:

# Example Usage
ticker = "NVDA"
option_data = download_option_data_xbbg(ticker)
print(option_data)
